In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: Microsoft.Data.Analysis"
#r "nuget: FSharp.Stats"

open FSharp.Data
open FSharp.Stats

let show a = printf "%A\n" a

type PriceStream = JsonProvider<"https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1h">

type Candle = 
    { Time : uint64
    ; Open : decimal
    ; High:  decimal
    ; Low : decimal
    ; Close: decimal }

type Vault = 
    { Time : uint64
    ; Collateral : decimal
    ; Debt : decimal
    ; Price : decimal
    ; TargetRatio : decimal
    ; LowerRatio : decimal
    ; UpperRatio : decimal
    } with
    member this.CollateralValue = this.Collateral * this.Price
    member this.ExcessCollateralValue = this.CollateralValue - this.Debt
    member this.ExcessCollateral = this.Collateral - (this.Debt / this.Price)
    member this.Ratio = this.CollateralValue / this.Debt
    member this.TargetLeverage = 1M/(this.TargetRatio - 1M) + 1M
    member this.ShouldRebalance = this.Ratio < this.LowerRatio || this.Ratio > this.UpperRatio
    member this.Rebalance = 
        // todo : add fees (gas, exchange, defisaver, slippage)
        let newCollateralValue = this.ExcessCollateralValue * this.TargetRatio
        let collateralValueDiff = newCollateralValue - this.CollateralValue
        let debtValueDiff = (collateralValueDiff / this.TargetRatio) * (this.TargetRatio - 1M)
        { this with 
              Collateral = this.Collateral + collateralValueDiff
            ; Debt = this.Debt + debtValueDiff
        }

    member this.NextVault (candle:Candle) = 
        let preRebalanceVault = { this with Time = candle.Time; Price = candle.Close }
        if not preRebalanceVault.ShouldRebalance then
            preRebalanceVault.Rebalance
        else
            preRebalanceVault

let create collateral time price targetRatio lowerRatio upperRatio = 
    let init = 
        { Time = time
        ; Collateral = collateral
        ; Debt = 0.0M
        ; Price = price
        ; TargetRatio = targetRatio
        ; LowerRatio = lowerRatio
        ; UpperRatio = upperRatio
        }
    init.Rebalance

let makeCandle (assetRow:(decimal array)) =
    { Time = (uint64)assetRow.[0]
    ; Open = assetRow.[1]
    ; High = assetRow.[2]
    ; Low = assetRow.[3]
    ; Close = assetRow.[4] 
    }

let assetHistory symbol interval (limit:int) =
    PriceStream.Load(String.Format("https://api.binance.com/api/v3/klines?symbol={0}&interval={1}&limit={2}", symbol, interval, limit))
    |> Array.map makeCandle

// makes a list of vaults over time that can be anylized 
let update (vaults: Vault list) (candle: Candle) =
    match vaults with
    | [] -> [create 100M candle.Time candle.Close 2.0M 1.7M 2.3M]
    | head::tail -> 
        // printf "%A" head
        [head.NextVault candle] @ vaults  

let ethPrices = assetHistory "ETHUSDT" "1h" 4000
ethPrices |> Array.fold update [] |> List.take 1

Installed Packages FSharp.Data, 4.2.4 FSharp.Stats, 0.4.2 Microsoft.Data.Analysis, 0.18.0

{ Time = 1631642400000UL
  Collateral = 335625.000000000M
  Debt = 167762.500000000M
  Price = 3355.25000000M
  TargetRatio = 2.0M
  LowerRatio = 1.7M
  UpperRatio = 2.3M }{ Time = 1631646000000UL
  Collateral = 335625.000000000M
  Debt = 167762.500000000M
  Price = 3350.11000000M
  TargetRatio = 2.0M
  LowerRatio = 1.7M
  UpperRatio = 2.3M }{ Time = 1631649600000UL
  Collateral = 335625.000000000M
  Debt = 167762.500000000M
  Price = 3374.93000000M
  TargetRatio = 2.0M
  LowerRatio = 1.7M
  UpperRatio = 2.3M }{ Time = 1631653200000UL
  Collateral = 335625.000000000M
  Debt = 167762.500000000M
  Price = 3345.77000000M
  TargetRatio = 2.0M
  LowerRatio = 1.7M
  UpperRatio = 2.3M }{ Time = 1631656800000UL
  Collateral = 335625.000000000M
  Debt = 167762.500000000M
  Price = 3419.54000000M
  TargetRatio = 2.0M
  LowerRatio = 1.7M
  UpperRatio = 2.3M }{ Time = 1631660400000UL
  Collateral = 335625.000000000M
  Debt = 167762.500000000M
  Price = 3434.37000000M
  TargetRatio = 2.0M
  LowerRa

index Time Collateral Debt Price TargetRatio LowerRatio UpperRatio CollateralValue ExcessCollateralValue ExcessCollateral Ratio TargetLeverage ShouldRebalance Rebalance 0 1635246000000 335625.000000000 167762.500000000 4209.83000000 2.0 1.7 2.3 1412924193.75000000000000000 1412756431.25000000000000000 335585.14981602582527085416751 8422.169398703524327546382535 2 True Time Collateral Debt Price TargetRatio LowerRatio UpperRatio CollateralValue ExcessCollateralValue ExcessCollateral Ratio TargetLeverage ShouldRebalance Rebalance 1635246000000 1412924293.750000000000000000 706462096.875000000000000000 4209.83000000 2.0 1.7 2.3 5948171079557.5625000000000000 5947464617460.6875000000000000 1412756481.250000000000000000 8419.660595903165735567970205 2 True { { Time = 1635246000000UL
 Collateral = 5948171079657.5625000000000000M
 Debt = 2974085539778.7812500000000000M
 Price = 4209.83000000M
 TargetRatio = 2.0M
 LowerRatio = 1.7M
 UpperRatio = 2.3M }: Time: 1635246000000, Collateral: 5948171079657.5625000000000000, Debt: 2974085539778.7812500000000000, Price: 4209.83000000, TargetRatio: 2.0, LowerRatio: 1.7, UpperRatio: 2.3, CollateralValue: 25040789056274796.339375000000, ExcessCollateralValue: 25037814970735017.558125000000, ExcessCollateral: 5947464617510.6875000000000000, Ratio: 8419.660000141550400743118371, TargetLeverage: 2, ShouldRebalance: True, Rebalance: { { Time = 1635246000000UL
 Collateral = 25040789056274896.339375000000M
 Debt = 12520394528137398.169687500000M
 Price = 4209.83000000M
 TargetRatio = 2.0M
 LowerRatio = 1.7M
 UpperRatio = 2.3M }: Time: 1635246000000, Collateral: 25040789056274896.339375000000, Debt: 12520394528137398.169687500000, Price: 4209.83000000, TargetRatio: 2.0, LowerRatio: 1.7, UpperRatio: 2.3, CollateralValue: 105417464992777746856.39105625, ExcessCollateralValue: 105404944598249609458.22136875, ExcessCollateral: 25037814970735067.558125000000, Ratio: 8419.66000000003362378070866, TargetLeverage: 2, ShouldRebalance: True, Rebalance: { { Time = 1635246000000UL
 Collateral = 105417464992777746956.39105625M
 Debt = 52708732496388873428.195528125M
 Price = 4209.83000000M
 TargetRatio = 2.0M
 LowerRatio = 1.7M
 UpperRatio = 2.3M }: Time: 1635246000000, Collateral: 105417464992777746956.39105625, Debt: 52708732496388873428.195528125, Price: 4209.83000000, TargetRatio: 2.0, LowerRatio: 1.7, UpperRatio: 2.3, CollateralValue: 443789606650545542469423.76033, ExcessCollateralValue: 443736897918049153595995.56480, ExcessCollateral: 105404944598249609508.22136875, Ratio: 8419.660000000000007986968763, TargetLeverage: 2, ShouldRebalance: True, Rebalance: { { Time = 1635246000000UL
 Collateral = 443789606650545542469523.76033M
 Debt = 221894803325272771234711.88017M
 Price = 4209.83000000M
 TargetRatio = 2.0M
 LowerRatio = 1.7M
 UpperRatio = 2.3M }: Time: 1635246000000, Collateral: 443789606650545542469523.76033, Debt: 221894803325272771234711.88017, Price: 4209.83000000, TargetRatio: 2.0, LowerRatio: 1.7, UpperRatio: 2.3, CollateralValue: 1868278799765666141054475212.0, ExcessCollateralValue: 1868056904962340868283240500.1, ExcessCollateral: 443736897918049153596045.56480, Ratio: 8419.660000000000000001897219, TargetLeverage: 2, ShouldRebalance: True, Rebalance: { { Time = 1635246000000UL
 Collateral = 1868278799765666141054475312.0M
 Debt = 934139399882833070527237606.0M
 Price = 4209.83000000M
 TargetRatio = 2.0M
 LowerRatio = 1.7M
 UpperRatio = 2.3M }: Time: 1635246000000, Collateral: 1868278799765666141054475312.0, Debt: 934139399882833070527237606.0, Price: 4209.83000000, TargetRatio: 2.0, LowerRatio: 1.7, UpperRatio: 2.3, CollateralValue: { System.OverflowException: Value was either too large or too small for a Decimal.
 at System.Number.ThrowOverflowException(TypeCode type)
 at System.Decimal.DecCalc.ScaleResult(Buf24* bufRes, UInt32 hiRes, Int32 scale)
 at System.Decimal.DecCalc.VarDecMul(DecCalc& d1, DecCalc& d2)
 at lambda_method607(Closure , Vault )
 at Microsoft.DotNet.Interactive.Formatting.Mem